In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import numpy as np 
import matplotlib.pyplot as plt
from IPython import display
import glob
import scipy

In [3]:
file_list = glob.glob('./data/*')

In [6]:
from src.mat_dataset import MAT_Dataset

In [7]:
ds = MAT_Dataset(file_list[0], [0,1])

In [10]:
x = next(iter(ds))

In [11]:
x.shape

torch.Size([1, 4, 64, 612])

In [8]:
mat = scipy.io.loadmat(file_list[0])

In [7]:
mat['Hfrq'].shape[1]

4

In [3]:
ue = 0
H = torch.from_numpy(mat['Hfrq'][0,ue][0,0][0])

In [5]:
H.shape

torch.Size([4, 64, 612, 100])